In [8]:
import pandas as pd
from datetime import datetime
import numpy as np 

In [9]:
df = pd.read_csv("../data/simulated/simulated_buses_multi.csv", parse_dates=["timestamp"])

In [10]:
df

df['timestamp'] = df['timestamp'].dt.floor('s')
grouped = df.groupby('timestamp')


In [11]:
df['manual_timediff'] = (df['timestamp'] - datetime(2025, 1, 12, 9, 0, 0)).dt.total_seconds().astype(int)

In [12]:
df

,timestamp,route_id,direction_id,latitude,longitude,bus_id,elapsed_seconds,manual_timediff
0,2025-01-12 09:00:00,292028,1,49.266003,-123.247825,636c2502-8780-46b5-8c5c-abfa8ae8a185,0,0
1,2025-01-12 09:01:10,292028,1,49.266138,-123.246883,636c2502-8780-46b5-8c5c-abfa8ae8a185,70,70
2,2025-01-12 09:02:34,292028,1,49.266323,-123.245606,636c2502-8780-46b5-8c5c-abfa8ae8a185,154,154
3,2025-01-12 09:03:46,292028,1,49.266351,-123.245407,636c2502-8780-46b5-8c5c-abfa8ae8a185,226,226
4,2025-01-12 09:04:16,292028,1,49.266364,-123.245318,636c2502-8780-46b5-8c5c-abfa8ae8a185,256,256
...,...,...,...,...,...,...,...,...
211195,2025-01-12 17:52:43,298165,0,49.266136,-123.245447,ad839414-3772-4acc-aeb4-50008edc2d19,9704,31963
211196,2025-01-12 17:54:00,298165,0,49.266410,-123.245678,ad839414-3772-4acc-aeb4-50008edc2d19,9781,32040
211197,2025-01-12 17:55:31,298165,0,49.267494,-123.246765,ad839414-3772-4acc-aeb4-50008edc2d19,9871,32131
211198,2025-01-12 17:56:01,298165,0,49.267331,-123.247308,ad839414-3772-4acc-aeb4-50008edc2d19,9901,32161


In [13]:
grouped_timediff = list(df['manual_timediff'].unique())

In [14]:
grouped_timediff.sort()

In [15]:
len(grouped_timediff)

31025

In [16]:
len(np.diff(grouped_timediff))

31024

In [1]:
import asyncio
import websockets
import json
from datetime import datetime

async def connect_to_websocket(speed_multiplier: float = 2.0):
    # Connect to the WebSocket server with speed multiplier
    uri = f"ws://localhost:8000/ws/simulated?speed_multiplier={speed_multiplier}"
    
    try:
        async with websockets.connect(uri) as websocket:
            print(f"Connected to WebSocket server (Speed: {speed_multiplier}x)")
            
            # Continuously receive messages
            while True:
                try:
                    # Receive and parse the JSON message
                    message = await websocket.recv()
                    data = json.loads(message)
                    
                    # Extract and format the timestamp for display
                    timestamp = datetime.fromisoformat(data['timestamp'].replace('Z', '+00:00'))
                    
                    # Print a formatted summary of the received data
                    print(f"\nTimestamp: {timestamp}")
                    print(f"Manual Time Difference: {data['manual_timediff']}")
                    print(f"Number of buses in this batch: {len(data['data'])}")
                    
                    # Print details of the first bus as a sample
                    if data['data']:
                        first_bus = data['data'][0]
                        print("\nSample bus data:")
                        print(f"Bus ID: {first_bus.get('bus_id')}")
                        print(f"Location: ({first_bus.get('latitude')}, {first_bus.get('longitude')})")
                        
                    print(data['data'])
                except websockets.exceptions.ConnectionClosed:
                    print("Connection closed by server")
                    break
                    
    except Exception as e:
        print(f"Error: {str(e)}")

# Since Jupyter requires special handling for asyncio, we'll use this helper function
async def run_test(speed_multiplier: float = 2.0):
    await connect_to_websocket(speed_multiplier)

# In Jupyter, you'll need to run this cell to start the connection
# Change the speed_multiplier value to adjust simulation speed
await run_test(speed_multiplier=20)  # Run at 2x speed

Connected to WebSocket server (Speed: 20x)

Timestamp: 2025-01-12 09:00:00
Manual Time Difference: 0
Number of buses in this batch: 2

Sample bus data:
Bus ID: 636c2502-8780-46b5-8c5c-abfa8ae8a185
Location: (49.266003, -123.247825)

Timestamp: 2025-01-12 09:01:10.348644
Manual Time Difference: 70
Number of buses in this batch: 1

Sample bus data:
Bus ID: 636c2502-8780-46b5-8c5c-abfa8ae8a185
Location: (49.2661378655, -123.2468828487)

Timestamp: 2025-01-12 09:01:46.294054
Manual Time Difference: 106
Number of buses in this batch: 1

Sample bus data:
Bus ID: bc1d2f55-b4f7-4dfc-bddd-9e3127e8a9de
Location: (49.2828118151, -123.1094909496)

Timestamp: 2025-01-12 09:02:34.354170
Manual Time Difference: 154
Number of buses in this batch: 1

Sample bus data:
Bus ID: 636c2502-8780-46b5-8c5c-abfa8ae8a185
Location: (49.266322521, -123.245605521)

Timestamp: 2025-01-12 09:02:56.014721
Manual Time Difference: 176
Number of buses in this batch: 1

Sample bus data:
Bus ID: bc1d2f55-b4f7-4dfc-bddd-9e3

CancelledError: 